In [ ]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
points_df = pd.read_csv("3d_points.csv")
points_df = points_df.sample(frac=1).reset_index(drop=True)
one_hot = {0:[1,0],1:[0,1]}
points_df["class"] = [one_hot[a] for a in points_df["class"]]
# print train_df.shape, test_df.shape

In [ ]:
train_df, test_df = train_test_split(points_df, test_size = 0.2)
train_data = train_df[["x","y","z"]].values
test_data = test_df[["x","y","z"]].values
print train_data.shape,test_data.shape

In [ ]:
train_labels = []
test_labels = []
for label in train_df["class"]:
    train_labels.append(label)
for label in test_df["class"]:
    test_labels.append(label)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
print train_labels.shape,test_labels.shape

In [ ]:
def plot_df(df):
    colors = {0:"r",1:"g"}
    c = [colors[a] for a in df["class"]]
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(df["x"],df["y"],df["z"],c=c)

In [ ]:
def plot_points(points,labels,epoch):
    colors = {0:"r",1:"g"}
#     print labels
    c = [colors[a] for a in labels]
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(points[:,0],points[:,1],points[:,2],c=c)
    plt.savefig("Epoch{}.png".format(epoch))

In [ ]:
X = tf.placeholder(tf.float32,[None,3])
Y = tf.placeholder(tf.float32)
num_h1_nodes = 3
num_h2_nodes = 4
n_classes = 2
batch_size=100
h1_layer = {"weights":tf.Variable(tf.random_normal([3,num_h1_nodes])),"biases":tf.Variable(tf.random_normal([num_h1_nodes]))}
h2_layer = {"weights":tf.Variable(tf.random_normal([num_h1_nodes,num_h2_nodes])),"biases":tf.Variable(tf.random_normal([num_h2_nodes]))}
op_layer = {"weights":tf.Variable(tf.random_normal([num_h2_nodes,n_classes])),"biases":tf.Variable(tf.random_normal([n_classes]))}
hl1 = tf.add(tf.matmul(X,h1_layer["weights"]),h1_layer["biases"])
hl1 = tf.nn.relu(hl1)

hl2 = tf.add(tf.matmul(hl1,h2_layer["weights"]),h2_layer["biases"])
hl2 = tf.nn.relu(hl2)

Y_pred = tf.add(tf.matmul(hl2,op_layer["weights"]),op_layer["biases"])
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_pred,labels=Y))

optimiser = tf.train.AdamOptimizer(learning_rate=0.02).minimize(cost)



num_epochs = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs):
        _,cost_val,weights = sess.run([optimiser,cost,h1_layer["weights"]],feed_dict={X:train_data,Y:train_labels})
        if epoch%(num_epochs/10)==0:
            print "Epoch: {}, Loss: {}, Weight: {}".format(epoch,cost_val,weights[0])
            correct = tf.equal(tf.argmax(Y_pred,1),tf.argmax(Y,1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print 'Accuracy:',accuracy.eval({X:test_data, Y:test_labels})
            test_output = sess.run(Y_pred,feed_dict={X:test_data, Y:test_labels})
            output_labels = np.array(test_output).argmax(axis=1)
#             print output_labels
            plot_points(test_data,output_labels,epoch)
    